In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('train.csv')
print(df.shape)
df.head()

(40000, 2)


,review,label
0,mature intelligent and highly charged melodram...,pos
1,http://video.google.com/videoplay?docid=211772...,pos
2,Title: Opera (1987) Director: Dario Argento Ca...,pos
3,I think a lot of people just wrote this off as...,pos
4,This is a story of two dogs and a cat looking ...,pos


In [3]:
from sklearn.preprocessing import LabelEncoder

In [4]:
le = LabelEncoder()
df['label'] = le.fit_transform(df.label.values)


In [5]:
df.head()

,review,label
0,mature intelligent and highly charged melodram...,1
1,http://video.google.com/videoplay?docid=211772...,1
2,Title: Opera (1987) Director: Dario Argento Ca...,1
3,I think a lot of people just wrote this off as...,1
4,This is a story of two dogs and a cat looking ...,1


In [6]:
X=df['review'].values
X.shape

(40000,)

In [38]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import string 

In [42]:
tokenizer=RegexpTokenizer('[a-zA-Z0-9]+')
ps=PorterStemmer()
sw=stopwords.words('english')
sw+=string.punctuation

In [43]:
def cleaning(doc):
    doc=doc.lower()
    doc=doc.replace('.<br /><br />',' ')
    
    tokens=tokenizer.tokenize(doc)
    new_token=[token for token in tokens if token not in sw]
    stoken=[ps.stem(token) for token in new_token]
    creview=' '.join(stoken)
    return creview

In [44]:
cleaning(X[1])


'http video googl com videoplay docid 211772166650071408 hl en distribut tri opt mass appeal want best possibl view rang forgo profit continu manual labor job gladli entertain work view texa tale pleas write like like alex like stuie texa texa tale write opinion rule'

In [45]:
X[1]

'http://video.google.com/videoplay?docid=211772166650071408&hl=en Distribution was tried.<br /><br />We opted for mass appeal.<br /><br />We want the best possible viewing range so, we forgo profit and continue our manual labor jobs gladly to entertain you for working yours.<br /><br />View Texas tale, please write about it... If you like it or not, if you like Alex or not, if you like Stuie, Texas or Texas tale... Just write about it.<br /><br />Your opinion rules.'

In [52]:
#from sklearn.feature_extraction.text import CountVectorizer

In [53]:
#cv=CountVectorizer()

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [47]:
tfidf=TfidfVectorizer()

In [48]:
inp=[]
for i in range(df.shape[0]):
    doc=cleaning(df.iloc[i][0])    
    inp.append(doc)

In [49]:
vec=tfidf.fit_transform(X)
vec.shape

(40000, 93813)

In [50]:
Y=df['label']


In [51]:
from sklearn.naive_bayes import MultinomialNB

In [52]:
mnb=MultinomialNB().fit(vec,Y)


In [53]:
mnb.score(vec,Y)

0.901425

In [54]:
test=pd.read_csv('test.csv')
test.shape


(10000, 1)

In [55]:
tes=[]
for i in range(test.shape[0]):
    doc=cleaning(df.iloc[i][0])    
    tes.append(doc)

In [56]:
vect=tfidf.transform(tes)

In [57]:
vect.shape

(10000, 93813)

In [58]:
vect[1].toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

In [59]:
o=mnb.predict(vect)

In [60]:
o.shape

(10000,)

In [61]:
ot=pd.DataFrame(data=o)

In [62]:
ot.head()

,0
0,1
1,1
2,1
3,1
4,1


In [63]:
ott=[]
def go(doc):
    for i in range(doc.shape[0]):
        if doc.iloc[i]==1:
            ott.append('pos')
        else:
            ott.append('neg')
    return ott        

In [64]:
ott=ot.apply(go)

In [65]:
ott.columns=['label']
ott

,label
0,pos
1,pos
2,pos
3,pos
4,pos
...,...
9995,neg
9996,neg
9997,pos
9998,pos


In [66]:
ind=np.arange(ott.shape[0])

In [67]:
ott.insert(0,'Id',ind)

In [68]:
ott.head()

,Id,label
0,0,pos
1,1,pos
2,2,pos
3,3,pos
4,4,pos


In [69]:
out=pd.DataFrame(ott)

In [70]:
out.to_csv('out.csv',index=False)